In [1]:
import glob
import os

import os
import sys
import argparse
import kenlm
import copy
from collections import namedtuple
import math
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.corpus import brown
import tqdm

In [2]:
# f= open('Encoder_decoder/outputs.txt', 'w')
# f1= open('Encoder_decoder/tgt_op.txt','w')
tgt_arr = []
op_arr = []

In [ ]:
#!/usr/bin/env python

"""
Version 0.21

Generate/re-order with an n-gram language model

An arpa formatted language model (as for example, a Kneser-Ney LM generated by
KenLM) is used to re-order a shuffled sentence. Optionally, a unigram LM can
be used for future costs. 

BNPs, if used, should be marked with:
    a start symbol: <sonp>
    an end symbol:  <eonp>

In the case of base NP symbols (BNPs) in the input, the n-gram models should be 
trained with BNPs. By default base NP symbols are treated as words. To
disable this, use the --no_npsyms_as_words flag, in which case the LMs should
be trained without BNP symbols, but the input can still contain BNPs, which
will still be correctly handled/scored in the beam decoder (but the BNP symbols
themselves will be ignored).

In the case of no base NPs in the input, the n-gram models should be trained
without base NP symbols. 

The KenLM Python bindings are required.

Neither the input nor the LM training files should contain explicit EOS symbols.

The input file must be pre-shuffled. For consistency with past work in this
line of research, in the case of BNPs, shuffling should treat BNPs as atomic 
units. To replicate our results, low-frequency tokens should be replaced with
special tokens and the post-processing script should be used to randomly 
replace these from remaining unused words before calculating BLEU. 
See the repo README for additional details.

The re-ordered output is printed to standard out.

"""


Hypothesis = namedtuple("Hypothesis", ['score', 'last_action', "bow", 
    "future_score", "state", "last_beam"])

def batch_advance(lm, inner_states, w, out_states):
    probs = []
    
    for state in inner_states:
        out_states.append(kenlm.State())
        probs.append(lm.BaseScore(state, w, out_states[-1]))
    
    return probs

def future(action, word_set, futurelm):
    """
    Pre-condition: action has been removed from word_set
    """
    score = 0
    if futurelm != {}:
        for words, c in word_set.items():
            if c == 0: continue
            for i, w in enumerate(words):
                score += futurelm[w] * c

    return score

def generate(lm, bow, beam_size, futurelm):
    
    n = sum([v*len(action) for action, v in bow.items()])
    start_state = kenlm.State()
    lm.BeginSentenceWrite(start_state)

    order = []
    beams = {}
    beams[0] = [Hypothesis(0, None, bow, future(None, bow, futurelm), 
        start_state, None)]
        
    for i in range(1, n+1):
        beams[i] = []

    for i in range(n):

        states = []
        
        for j, hyp in enumerate(beams[i]):
            states.append(hyp.state)
            
        for action in bow:
            # Advance
            inner_states = states
            scores = [0.0] * len(beams[i])
            
            for w in action:
                out_states = []
                new_scores = batch_advance(lm, inner_states, w, out_states)
                inner_states = out_states
                for new_score_i, new_score in enumerate(new_scores):
                    scores[new_score_i] += new_score

            # Add to beam.
            ni = i + len(action)
            
            for j, hyp in enumerate(beams[i]):
                score = scores[j]
                out_state = inner_states[j]
                
                if hyp.bow[action] == 0:
                    continue
                
                new_bow = copy.copy(hyp.bow)
                new_bow[action] -= 1
                fscore = future(action, new_bow, futurelm)               
                if len(beams[ni]) < beam_size or (hyp.score+score+fscore
                    > beams[ni][-1].score + beams[ni][-1].future_score):
 
                    new_hyp = Hypothesis(hyp.score+score, action, new_bow, 
                        fscore, out_state, j)
                    beams[ni].append(new_hyp)
                    beams[ni].sort(key=lambda a: a.score + a.future_score)
                    beams[ni].reverse()
                    beams[ni] = beams[ni][:beam_size]

    cur = n
    pos = 0
    while cur > 0:
        order.extend(reversed(beams[cur][pos].last_action))
        old_cur = cur
        cur -= len(beams[cur][pos].last_action)
        pos = beams[old_cur][pos].last_beam

    order.reverse()
    return order

def main(arguments):
        brown_sent = brown.sents()
        i=0
        parser = argparse.ArgumentParser(
            description=__doc__,
            formatter_class=argparse.RawDescriptionHelpFormatter)

        parser.add_argument('lm', help="Language model", type=str)    
        parser.add_argument('test', help="Shuffled file (one sentence per line, no \
            EOS symbols, to re-order.", type=str)
        parser.add_argument('beamsize', help="Beam size to use.", type=int)
        parser.add_argument('-f', '--future', help="LM for unigram future costs. \
            If omitted, future costs are not calculated.", type=str, default="")

        parser.add_argument('-n', '--no_npsyms_as_words',
            help="Do not treat base NP symbols as words.", action="store_true")

        args = parser.parse_args(arguments)
        lm = kenlm.Model(args.lm)

        futurelm = {}
        if args.future != "":
            for l in open(args.future):
                t = l.strip().split()
                if len(t) == 2 and t[0] != "ngram":        
                    futurelm[t[1]] = float(t[0])
    #     print(args.test)
        for l in open(args.test):
    #         print(l)
            bow = {}

            print(l)
            in_bnp = False
            cur_bnp = []
            for w in l.strip().split():
                if w == "<sonp>":
                    in_bnp = True
                    cur_bnp = []
                    continue
                if w == "<eonp>":
                    in_bnp = False
                    if not args.no_npsyms_as_words:
                        cur_bnp = ["<sonp>"] + cur_bnp + ["<eonp>"]
                    write = tuple(cur_bnp)
                else:
                    write = (w,)

                if in_bnp:
                    cur_bnp.append(w)
                    continue

                bow.setdefault(write, 0)
                bow[write] += 1
            try:
                sen = (" ".join(generate(lm, bow, args.beamsize, futurelm)))
                tgt_arr.append(brown_sent[i]) 
            except:
                pass
            i+=1
#             f.writeline(sen)
#             break

In [ ]:
# if __name__ == '__main__':
#     sys.exit(main(sys.argv[1:]))
arg =['5_gram_model.arpa', 'Encoder_decoder/shuffle.txt', '64', '--future', 'future_model.arpa']
main(arg)

In [ ]:
# f= open('Encoder_decoder/shuffle.txt','r')

In [ ]:
# for sent in f :

In [6]:
def removeChar(s, c) : 
      
    # find total no. of  
    # occurence of charcter 
    counts = s.count(c) 
  
    # convert into list  
    # of characters 
    s = list(s) 
  
    # keeep looping untill  
    # counts become 0 
    while counts : 
          
        # remove character 
        # from the list 
        s.remove(c) 
  
        # decremented by one 
        counts -= 1
  
    # join all remaining characters 
    # of the list with empty string  
    s = '' . join(s) 
      
    return s 
 

In [ ]:
 removeChar("'' Atlanta's County Friday Fulton Grand Jury The `` an any election evidence investigation irregularities no of place primary produced recent said that took . ","'")

In [ ]:
' '.join(word for word in brown.sents()[555])

In [4]:
#!/usr/bin/env python

"""
Version 0.21

Generate/re-order with an n-gram language model

An arpa formatted language model (as for example, a Kneser-Ney LM generated by
KenLM) is used to re-order a shuffled sentence. Optionally, a unigram LM can
be used for future costs. 

BNPs, if used, should be marked with:
    a start symbol: <sonp>
    an end symbol:  <eonp>

In the case of base NP symbols (BNPs) in the input, the n-gram models should be 
trained with BNPs. By default base NP symbols are treated as words. To
disable this, use the --no_npsyms_as_words flag, in which case the LMs should
be trained without BNP symbols, but the input can still contain BNPs, which
will still be correctly handled/scored in the beam decoder (but the BNP symbols
themselves will be ignored).

In the case of no base NPs in the input, the n-gram models should be trained
without base NP symbols. 

The KenLM Python bindings are required.

Neither the input nor the LM training files should contain explicit EOS symbols.

The input file must be pre-shuffled. For consistency with past work in this
line of research, in the case of BNPs, shuffling should treat BNPs as atomic 
units. To replicate our results, low-frequency tokens should be replaced with
special tokens and the post-processing script should be used to randomly 
replace these from remaining unused words before calculating BLEU. 
See the repo README for additional details.

The re-ordered output is printed to standard out.

"""


Hypothesis = namedtuple("Hypothesis", ['score', 'last_action', "bow", 
    "future_score", "state", "last_beam"])

def batch_advance(lm, inner_states, w, out_states):
    probs = []
    
    for state in inner_states:
        out_states.append(kenlm.State())
        probs.append(lm.BaseScore(state, w, out_states[-1]))
    
    return probs

def future(action, word_set, futurelm):
    """
    Pre-condition: action has been removed from word_set
    """
    score = 0
    if futurelm != {}:
        for words, c in word_set.items():
            if c == 0: continue
            for i, w in enumerate(words):
                score += futurelm[w] * c

    return score

def generate(lm, bow, beam_size, futurelm):
    
    n = sum([v*len(action) for action, v in bow.items()])
    start_state = kenlm.State()
    lm.BeginSentenceWrite(start_state)

    order = []
    beams = {}
    beams[0] = [Hypothesis(0, None, bow, future(None, bow, futurelm), 
        start_state, None)]
        
    for i in range(1, n+1):
        beams[i] = []

    for i in range(n):

        states = []
        
        for j, hyp in enumerate(beams[i]):
            states.append(hyp.state)
            
        for action in bow:
            # Advance
            inner_states = states
            scores = [0.0] * len(beams[i])
            
            for w in action:
                out_states = []
                new_scores = batch_advance(lm, inner_states, w, out_states)
                inner_states = out_states
                for new_score_i, new_score in enumerate(new_scores):
                    scores[new_score_i] += new_score

            # Add to beam.
            ni = i + len(action)
            
            for j, hyp in enumerate(beams[i]):
                score = scores[j]
                out_state = inner_states[j]
                
                if hyp.bow[action] == 0:
                    continue
                
                new_bow = copy.copy(hyp.bow)
                new_bow[action] -= 1
                fscore = future(action, new_bow, futurelm)               
                if len(beams[ni]) < beam_size or (hyp.score+score+fscore
                    > beams[ni][-1].score + beams[ni][-1].future_score):
 
                    new_hyp = Hypothesis(hyp.score+score, action, new_bow, 
                        fscore, out_state, j)
                    beams[ni].append(new_hyp)
                    beams[ni].sort(key=lambda a: a.score + a.future_score)
                    beams[ni].reverse()
                    beams[ni] = beams[ni][:beam_size]

    cur = n
    pos = 0
    while cur > 0:
        order.extend(reversed(beams[cur][pos].last_action))
        old_cur = cur
        cur -= len(beams[cur][pos].last_action)
        pos = beams[old_cur][pos].last_beam

    order.reverse()
    return order

def main(arguments):
        brown_sent = brown.sents()
        i=0
        parser = argparse.ArgumentParser(
            description=__doc__,
            formatter_class=argparse.RawDescriptionHelpFormatter)

        parser.add_argument('lm', help="Language model", type=str)    
        parser.add_argument('test', help="Shuffled file (one sentence per line, no \
            EOS symbols, to re-order.", type=str)
        parser.add_argument('beamsize', help="Beam size to use.", type=int)
        parser.add_argument('-f', '--future', help="LM for unigram future costs. \
            If omitted, future costs are not calculated.", type=str, default="")

        parser.add_argument('-n', '--no_npsyms_as_words',
            help="Do not treat base NP symbols as words.", action="store_true")

        args = parser.parse_args(arguments)
        lm = kenlm.Model(args.lm)

        futurelm = {}
        if args.future != "":
            for l in open(args.future):
                t = l.strip().split()
                if len(t) == 2 and t[0] != "ngram":        
                    futurelm[t[1]] = float(t[0])
    #     print(args.test)
        for l in open(args.test):
    #         print(l)
            bow = {}

#             print(l)
            in_bnp = False
            cur_bnp = []
            for w in l.strip().split():
                if w == "<sonp>":
                    in_bnp = True
                    cur_bnp = []
                    continue
                if w == "<eonp>":
                    in_bnp = False
                    if not args.no_npsyms_as_words:
                        cur_bnp = ["<sonp>"] + cur_bnp + ["<eonp>"]
                    write = tuple(cur_bnp)
                else:
                    write = (w,)

                if in_bnp:
                    cur_bnp.append(w)
                    continue

                bow.setdefault(write, 0)
                bow[write] += 1
            try:
                sen = (" ".join(generate(lm, bow, args.beamsize, futurelm)))
                tgt_arr.append(brown_sent[i])
                op_arr.append(sen)
            except:
                pass
            i+=1
            
            if(i%1000 == 0):
                print(i)
            if(i==10000):#Number of sentences linearized = 10000, Change the number Manually
                break
#             f.writeline(sen)
#             break

In [5]:
# arg =['5_gram_model.arpa', 'Encoder_decoder/untitled.txt', '64', '--future', 'future_model.arpa']
arg =['language_models/5_gram_model.arpa', 'data/shuffled.txt', '64', '--future', 'language_models/future_model.arpa']
main(arg)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [6]:
# len(tgt_arr)

5718

In [7]:
# len(op_arr)

5718

In [ ]:
# tgt_arr[498]

In [ ]:
file = open('outputs.txt')
file.readlines(op_arr)
file.close()

## To calculate ROUGE scores

In [10]:
from PyRouge.pyrouge import Rouge

In [11]:
f_scores=[]
r = Rouge()

In [12]:
for i in tqdm.tqdm(range(len(op_arr))):
    if op_arr[i]:
        op_sent = op_arr[i]
        tgt_sent = ' '.join(word for word in tgt_arr[i])
        _,_,score = r.rouge_l([op_sent], [tgt_sent])
#         print(i)
        f_scores.append(score)

100%|██████████| 5718/5718 [01:04<00:00, 88.49it/s]


In [16]:
print('The average F-score for sentences is ',sum(f_scores)/len(f_scores))

0.8050810835986757

In [17]:
import statistics

In [18]:
print('The std dev of F-scores for sentences is ',sum(f_scores)/len(f_scores))

0.16018926743746137

In [95]:
len(f_scores)

2173